In [1]:
# импорт библиотек

# предобработка

import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm

# модель

from catboost import CatBoostClassifier, Pool, cv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

import joblib
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import cross_val_score
import pickle
import os

In [2]:
#читает паркет
parquet_file = pq.ParquetFile('D:\\hack_yandex\\dataset_train.parquet')

#задаем размер батча
batch_size = 125000

# выьираем индекс батча,в дальнейшем можно указать индекс (2-3 и т.д. это даст новые даннные для дообучениЯ моедли)
target_index = 1

# цикл
for index, batch in enumerate(parquet_file.iter_batches(batch_size=batch_size)):
    if index == target_index:
        df = batch.to_pandas()
        print("Batch Index:", index)
        print("batch_df length:", len(df))
        break
else:
    print(f"Батч с индексом {target_index} не найден.")

Batch Index: 1
batch_df length: 125000


In [3]:
display(df.info())
display(f"Количество пропусков: {df.isna().sum().sum()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Columns: 2780 entries, id to communication_availability_69_1_flg
dtypes: float64(2607), int32(121), int64(31), int8(19), object(2)
memory usage: 2.5+ GB


None

'Количество пропусков: 61462641'

In [4]:
#заполнение пропусков медианой по chanel name
# tqdm для отображения процесса

tqdm.pandas()

columns_with_nan = df.columns[df.isnull().any()].tolist()

def fillna_grouped(group):
    if group.notnull().any():
        return group.fillna(group.median())
    else:
        return group

for column in tqdm(columns_with_nan):
    df[column] = df.groupby('channel_name')[column].progress_apply(fillna_grouped)
    
display(f"Количество пропусков: {df.isna().sum().sum()}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 97.97it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 124.90it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 124.90it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 128.23it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 128.22it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 121.95it/s]

100%|█████████████████████████████████████████████████████

'Количество пропусков: 2563804'

In [5]:
#заполняем нан 0
df=df.fillna(0)
display(f"Количество пропусков: {df.isna().sum().sum()}")

'Количество пропусков: 0'

In [6]:
label_encoder = LabelEncoder()
df['channel_name_encoded'] = label_encoder.fit_transform(df['channel_name'])

In [7]:
# разделение на фичи
X = df.drop(columns=['target'])
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [14]:
# модель 
model = CatBoostClassifier()

params = {
    'loss_function': ['Logloss',], # Функция потерь
    'iterations': [2000,2500], # итерации 
    'depth': [8,10], # глубина
    'learning_rate': [0.01, 0.05, 0.1], # шаг
    'task_type': ['GPU'], # инициализация граф процессора
    'devices': ['0:1'], # выбор граф процесса
    'verbose': [100],  # сколько шагов отображать
    'cat_features': [['channel_name', 'id']], # категорилаьные признаки
    'early_stopping_rounds': [100] 
}
roc_auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

In [15]:
grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5,scoring=roc_auc_scorer)

#GПоиск по стеке
grid_search.fit(X_train, y_train, cat_features=['channel_name', 'id'])

print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая оценка на тренировочном наборе данных (ROC AUC):", grid_search.best_score_)

# Оценка
test_roc_auc = grid_search.score(X_test, y_test)
print("Оценка лучшей модели на тестовом наборе данных (ROC AUC):", test_roc_auc)

0:	learn: 0.6692747	total: 95ms	remaining: 3m 57s
100:	learn: 0.0658288	total: 6.5s	remaining: 2m 34s
200:	learn: 0.0442926	total: 12.9s	remaining: 2m 27s
300:	learn: 0.0416787	total: 19.2s	remaining: 2m 20s
400:	learn: 0.0408438	total: 25.4s	remaining: 2m 12s
500:	learn: 0.0402421	total: 31.5s	remaining: 2m 5s
600:	learn: 0.0397334	total: 38.1s	remaining: 2m
700:	learn: 0.0393021	total: 44.6s	remaining: 1m 54s
800:	learn: 0.0388596	total: 50.9s	remaining: 1m 47s
900:	learn: 0.0384380	total: 57.1s	remaining: 1m 41s
1000:	learn: 0.0380149	total: 1m 3s	remaining: 1m 34s
1100:	learn: 0.0375932	total: 1m 9s	remaining: 1m 28s
1200:	learn: 0.0371816	total: 1m 16s	remaining: 1m 22s
1300:	learn: 0.0367605	total: 1m 22s	remaining: 1m 16s
1400:	learn: 0.0363303	total: 1m 29s	remaining: 1m 9s
1500:	learn: 0.0359148	total: 1m 35s	remaining: 1m 3s
1600:	learn: 0.0354871	total: 1m 42s	remaining: 57.3s
1700:	learn: 0.0350953	total: 1m 48s	remaining: 50.9s
1800:	learn: 0.0346742	total: 1m 54s	remainin

In [66]:
best_model = grid_search.best_estimator_
best_model.save_model('feature_model.cbm')

In [67]:
feature_importances = best_model.get_feature_importance(prettified=True)
non_zero_importances = feature_importances[feature_importances['Importances'] !=0]
non_zero_importances

,Feature Id,Importances
0,communication_availability_30_1_flg,3.545749
1,channel_name,3.493448
2,materials_details_16_1_ctg,2.524644
3,materials_details_6_1_num,2.361640
4,tariff_plans_8_1_num,2.050425
...,...,...
1764,user_devices_15_1_cnt,0.000066
1765,payments_details_41_6_sum,0.000057
1766,spas_symptoms_agr_245_6_avg,0.000042
1767,markers_356_1_cnt,0.000039


In [10]:
selected_features = non_zero_importances['Feature Id'].values

# селеккция
X_selected = X[selected_features]
y = df['target']
X_train, X_val, y_train, y_val = train_test_split(X_selected, y, test_size=0.25, random_state=42)

In [11]:
saved_model = saved_model.load_model('feature_model.cbm')

In [14]:
cv_scores = cross_val_score(saved_model, X_selected, y, cv=5, scoring='roc_auc')
print("Среднее значение ROC AUC на кросс-валидации: {:.2f}".format(cv_scores.mean()))

final_model = saved_model
final_model.fit(X_selected, y, verbose=200)


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.6685327	total: 55.5ms	remaining: 2m 18s
100:	learn: 0.0667670	total: 3.98s	remaining: 1m 34s
200:	learn: 0.0451983	total: 7.61s	remaining: 1m 27s
300:	learn: 0.0427492	total: 11.2s	remaining: 1m 21s
400:	learn: 0.0420131	total: 14.8s	remaining: 1m 17s
500:	learn: 0.0415201	total: 18.3s	remaining: 1m 13s
600:	learn: 0.0411241	total: 21.9s	remaining: 1m 9s
700:	learn: 0.0407436	total: 25.5s	remaining: 1m 5s
800:	learn: 0.0404007	total: 29.2s	remaining: 1m 1s
900:	learn: 0.0400586	total: 32.8s	remaining: 58.2s
1000:	learn: 0.0397282	total: 36.4s	remaining: 54.6s
1100:	learn: 0.0393951	total: 40.1s	remaining: 50.9s
1200:	learn: 0.0390519	total: 43.8s	remaining: 47.3s
1300:	learn: 0.0387129	total: 47.4s	remaining: 43.7s
1400:	learn: 0.0383817	total: 51s	remaining: 40s
1500:	learn: 0.0380421	total: 54.6s	remaining: 36.3s
1600:	learn: 0.0377029	total: 58.2s	remaining: 32.7s
1700:	learn: 0.0373492	total: 1m 1s	remaining: 29s
1800:	learn: 0.0369991	total: 1m 5s	remaining: 25.4s
1900

In [15]:
y_pred_proba = final_model.predict_proba(X_selected)[:, 1]

# Влучшая модель
roc_auc_final = roc_auc_score(y, y_pred_proba)
print("ROC AUC для лучшей модели: {:.2f}".format(roc_auc_final))

ROC AUC для лучшей модели: 0.87


In [18]:
final_model.save_model('feature_model_final.cbm')

In [19]:
test_df = pq.read_table('D:\\hack_yandex\\features_oot.parquet').to_pandas()

In [20]:
model = model.load_model('feature_model_final.cbm')

In [21]:
tqdm.pandas()

columns_with_nan = test_df.columns[test_df.isnull().any()].tolist()

def fillna_grouped(group):
    if group.notnull().any():
        return group.fillna(group.median())
    else:
        return group

for column in tqdm(columns_with_nan):
    test_df[column] = test_df.groupby('channel_name')[column].progress_apply(fillna_grouped)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 47.60it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 66.41it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 54.63it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 71.44it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 68.51it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 47.63it/s]

100%|█████████████████████████████████████████████████████

In [24]:
label_encoder = LabelEncoder()
test_df['channel_name_encoded'] = label_encoder.fit_transform(test_df['channel_name'])

In [26]:
probabilities = model.predict_proba(test_df)[:, 1]


In [27]:
probabilities

array([0.00657889, 0.00644591, 0.0066974 , ..., 0.00735672, 0.00838372,
       0.00424155])

In [29]:
predictions_df = pd.DataFrame({'id': range(len(predictions)), 'predictions': probabilities})

# сохранения файла
predictions_df.to_csv('predictions_catboost_new.csv', index=False)


In [30]:
df_3=pd.read_csv('predictions_catboost_new.csv')
df_3

,id,predictions
0,0,0.006579
1,1,0.006446
2,2,0.006697
3,3,0.004109
4,4,0.008795
...,...,...
60656,60656,0.004339
60657,60657,0.009799
60658,60658,0.007357
60659,60659,0.008384
